In [1]:
import datetime
import math
import pprint
import time

import ee
from IPython.display import Image

import openet
import openet.disalexi.disalexi as disalexi
import openet.disalexi.landsat as landsat
import openet.disalexi.utils as utils

ee.Initialize()

### ALEXI Grid Properties

In [2]:
alexi_coll = ee.ImageCollection('projects/disalexi/alexi/CONUS')
# print(ee.Image(alexi_coll.first()).projection().getInfo()['transform'])
# print(ee.Image(alexi_coll.first()).projection().getInfo()['crs'])
# print(ee.Image(alexi_coll.first()).getInfo()['bands'][0]['dimensions'])
alexi_cs = 0.04
alexi_geo = [0.04, 0.0, -125.04, 0.0, -0.04, 49.82]
alexi_shape = [1456, 625]
alexi_crs = 'EPSG:4326'
alexi_extent = [
    alexi_geo[2], alexi_geo[5] + alexi_geo[4] * alexi_shape[1], 
    alexi_geo[2] + alexi_geo[0] * alexi_shape[0], alexi_geo[5]]
alexi_geo_str = '[' + ','.join(list(map(str, alexi_geo))) + ']'
alexi_shape_str = '{0}x{1}'.format(*alexi_shape)
# print(alexi_geo_str)
# print(alexi_shape_str)

### Study Area Properties

In [3]:
output_crs = 'EPSG:4326'
output_cs = 0.04   # ALEXI cellsize

# # Study areas
# output_extent = [-121.9, 38.8, -121.7, 38.9]  # Study Area
# output_extent = [-122.0, 38.7, -121.6, 39.0]  # Study Area
# output_extent = [-122.0, 38.0, -121.0, 39.0]  # 1 x 1 deg
output_extent = [-123.0, 37.8, -120.4, 40.0]  # LC08_044033_20150711
# output_extent = [-123, 35, -118.5, 40]  # Central Valley
# output_extent = [-125, 32, -114, 42]  # California / Nevada
# output_extent = [-125, 25, -65, 50]  # CONUS

# Computed output transform, extent, and shape
output_geom = ee.Geometry.Rectangle(output_extent, output_crs, False)
output_region = output_geom.bounds(1, output_crs).coordinates().getInfo()[0][:-1]
output_xmin = min(x for x, y in output_region)
output_ymin = min(y for x, y in output_region)
output_xmax = max(x for x, y in output_region)
output_ymax = max(y for x, y in output_region)
# Expand extent when snapping/aligning to ALEXI grid
output_xmin = math.floor((output_xmin - alexi_extent[0]) / output_cs) * output_cs + alexi_extent[0]
output_ymin = math.floor((output_ymin - alexi_extent[3]) / output_cs) * output_cs + alexi_extent[3]
output_xmax = math.ceil((output_xmax - alexi_extent[0]) / output_cs) * output_cs + alexi_extent[0]
output_ymax = math.ceil((output_ymax - alexi_extent[3]) / output_cs) * output_cs + alexi_extent[3]
output_extent = [output_xmin, output_ymin, output_xmax, output_ymax]
output_geo = [output_cs, 0.0, output_xmin, 0.0, -output_cs, output_ymax]

# Convert to strings for export calls
output_geo_str = '[' + ','.join(list(map(str, output_geo))) + ']'
output_shape_str = '{0}x{1}'.format(
    int(abs(output_extent[2] - output_extent[0]) / output_cs),
    int(abs(output_extent[3] - output_extent[1]) / output_cs))
output_cs_str = '{:0.3f}'.format(output_cs).replace('.', 'p')
print(output_geo_str)
print(output_shape_str)
print(output_cs_str)

[0.04,0.0,-123.0,0.0,-0.04,40.019999999999996]
65x55
0p040


### Landsat Image, Collection, and Properties

In [4]:
landsat_coll_id = 'LANDSAT/LC08/C01/T1_RT_TOA'
landsat_id = 'LC08_044033_20150711'
landsat_img = ee.Image('LANDSAT/LC08/C01/T1_RT_TOA/LC08_044033_20150711')

landsat_crs = landsat_img.select(['B2']).projection().getInfo()['crs']
landsat_geo = landsat_img.select(['B2']).projection().getInfo()['transform']
landsat_shape = landsat_img.select(['B2']).getInfo()['bands'][0]['dimensions']
landsat_geo_str = '[' + ','.join(list(map(str, landsat_geo))) + ']'
landsat_shape_str = '{0}x{1}'.format(*landsat_shape)
print(landsat_crs)
print(landsat_geo_str)
print(landsat_shape_str)

EPSG:32610
[30.0,0.0,499785.0,0.0,-30.0,4423215.0]
7671x7791


In [5]:
# Eventually try mapping the functions over a collection of images
landsat_coll = ee.ImageCollection(landsat_coll_id) \
    .filterDate('2015-07-11', '2015-07-12') \
    .filterBounds(output_geom) \
    .filterMetadata('CLOUD_COVER_LAND', 'less_than', 70)
    # .filterMetadata('WRS_PATH', 'equals', 44) \
    # .filterMetadata('WRS_ROW', 'not_greater_than', 34) \
    # .filterMetadata('WRS_ROW', 'not_less_than', 33) \
# pprint.pprint(list(landsat_coll.aggregate_histogram('system:index').getInfo().keys()))

### Ta Function

In [6]:
def get_affine_transform(image):
    return ee.List(ee.Dictionary(ee.Algorithms.Describe(image.projection())).get('transform'))

def ta_landsat_func(l_img):
    """Compute air temperature at the Landsat scale (don't aggregate)"""
    input_img = ee.Image(landsat.Landsat(l_img).prep())

    # Use the CONUS ALEXI ET but the global landcover and elevation products
    d_obj = disalexi.Image(
        input_img, 
        iterations=10,
        elevation=ee.Image('USGS/SRTMGL1_003').rename(['elevation']),
        landcover=ee.Image(ee.ImageCollection('users/cgmorton/GlobeLand30')
                               .filterBounds(l_img.geometry().bounds(1)).mosaic()) \
            .divide(10).floor().multiply(10).rename(['landcover']),
        lc_type='GLOBELAND30',
        tair_values=list(range(273, 321, 1)),
    )

    return d_obj.compute_ta()

    # return d_obj.compute_ta().reproject(
    #     crs=landsat_img.select(['B2']).projection().crs(), 
    #     crsTransform=get_affine_transform(landsat_img.select(['B2'])))
    
    
def ta_coarse_func(l_img):
    """Compute air temperature averaged/aggregated to the ALEXI grid"""
    input_img = ee.Image(landsat.Landsat(l_img).prep())

    # Use the CONUS ALEXI ET but the global landcover and elevation products
    d_obj = disalexi.Image(
        input_img, 
        iterations=10,
        elevation=ee.Image('USGS/SRTMGL1_003').rename(['elevation']),
        landcover=ee.Image(ee.ImageCollection('users/cgmorton/GlobeLand30')
                           .filterBounds(l_img.geometry().bounds(1)).mosaic()) \
            .divide(10).floor().multiply(10).rename(['landcover']),
        lc_type='GLOBELAND30',
        tair_values=list(range(273, 321, 1)),
    )
    
    # Was testing to see if setting the crsTransform in the reproject helped (it didn't)
    landsat_crs = landsat_img.select(['B2']).projection().crs()
    # landsat_geo = get_affine_transform(l_img.select(['B2']))
    # .reproject(crs=landsat_crs, crsTransform=landsat_geo)\
    
    return d_obj.compute_ta()\
        .reproject(crs=landsat_crs, scale=30)\
        .reduceResolution(reducer=ee.Reducer.mean(), maxPixels=65535) \
        .reproject(crs=output_crs, crsTransform=output_geo) \
        .updateMask(1)

### Ta Export - Aggregated/averaged to ALEXI grid

Export the aggregated air temperature asset.  This fails with an internal error.

In [7]:
export_id = 'LC08_044033_20150711'
export_img = ee.Image('{}/{}'.format('LANDSAT/LC08/C01/T1_RT_TOA', export_id))

asset_id = '{coll}/{image}_{cs}'.format(coll='projects/disalexi/ta/CONUS', image=export_id, cs=output_cs_str)
task_id = 'disalexi_tair_coarse_{image}_{cs}'.format(image=export_id, cs=output_cs_str)
ta_coarse_img = ta_coarse_func(export_img)\
    .setMulti({
        'DATE_INGESTED': datetime.datetime.now().strftime('%Y-%m-%d'),
        'DISALEXI_VERSION': openet.disalexi.__version__,
        'DATE': ee.Date(export_img.get('system:time_start')).format('YYYY-mm-dd'),
    })

task = ee.batch.Export.image.toAsset(
    image=ee.Image(ta_coarse_img).toFloat(),
    description=task_id,
    assetId=asset_id,
    crs=output_crs,
    crsTransform=output_geo_str,
    dimensions=output_shape_str,
)
# task.start()
# time.sleep(1)
# print(task.status())

### Ta Export - Landsat Scale (30 or 60m)

Export the Landsat scale air temperature asset.  This completes successfully for 30 or 60m cellsize.

In [8]:
export_id = 'LC08_044033_20150711'
export_img = ee.Image('{}/{}'.format('LANDSAT/LC08/C01/T1_RT_TOA', export_id))

# Switch output cellsize to 60m
export_cs = 30
export_crs = export_img.select(['B2']).projection().getInfo()['crs']
export_geo = export_img.select(['B2']).projection().getInfo()['transform']
export_geo[0] = export_cs
export_geo[4] = -export_cs
export_shape = export_img.select(['B2']).getInfo()['bands'][0]['dimensions']
export_shape[0] = int(export_shape[0] / (export_cs / 30) + 0.5)
export_shape[1] = int(export_shape[1] / (export_cs / 30) + 0.5)
# print(export_crs)
# print(export_geo)
# print(export_shape)

asset_id = '{coll}_{cs}m/{image}'.format(
    coll='projects/disalexi/ta/landsat', image=export_id, cs=export_cs)
task_id = 'disalexi_tair_landsat_{image}_{cs}m'.format(image=export_id, cs=export_cs)

ta_landsat_img = ta_landsat_func(export_img)\
    .setMulti({
        'DATE_INGESTED': datetime.datetime.now().strftime('%Y-%m-%d'),
        'DISALEXI_VERSION': openet.disalexi.__version__,
        'DATE': ee.Date(export_img.get('system:time_start')).format('YYYY-mm-dd'),
    })
task = ee.batch.Export.image.toAsset(
    image=ee.Image(ta_landsat_img).toFloat(),
    description=task_id,
    assetId=asset_id,
    crs=export_crs,
    crsTransform='[' + ','.join(list(map(str, export_geo))) + ']',
    dimensions='{0}x{1}'.format(*export_shape),
)
# task.start()
# time.sleep(1)
# print(task.status())

### Ta Export - Mosaiced Landsat Images (for one WRS path, date, UTM zone)

Start with a small mosaic that is only 2 or 3 images in the same path.  Eventually the exports may need to be for all images in the path or by path and UTM zone.

In [9]:
# export_id = 'LC08_044033_20150711'
# export_img = ee.Image('{}/{}'.format(l8_coll_id, export_id))

# Try calling the function for a mosaiced a collection of images in the same UTM zone, WRS Path, and date
export_coll = ee.ImageCollection('LANDSAT/LC08/C01/T1_RT_TOA') \
    .filterDate('2015-07-11', '2015-07-12') \
    .filterMetadata('WRS_PATH', 'equals', 44) \
    .filterMetadata('WRS_ROW', 'not_greater_than', 34) \
    .filterMetadata('WRS_ROW', 'not_less_than', 33) \
    .filterMetadata('CLOUD_COVER_LAND', 'less_than', 70)
# print(export_coll.aggregate_histogram('system:index').getInfo())

export_id = '20150711_p044'
export_img = landsat_coll.mean()\
    .set('system:time_start', ee.Date.fromYMD(2015, 7, 11).millis())
landsat_crs = 'EPSG:32610'
landsat_cs = 60

def ta_func(l_img):
    input_img = ee.Image(landsat.Landsat(l_img).prep())

    # Use the CONUS ALEXI ET but the global landcover and elevation products
    d_obj = disalexi.Image(
        input_img, 
        iterations=10,
        elevation=ee.Image('USGS/SRTMGL1_003').rename(['elevation']),
        landcover=ee.Image(ee.ImageCollection('users/cgmorton/GlobeLand30')
                           .filterBounds(l_img.geometry().bounds(1)).mosaic()) \
            .divide(10).floor().multiply(10).rename(['landcover']),
        lc_type='GLOBELAND30',
        tair_values=list(range(273, 321, 1)),
    )
    
    return d_obj.compute_ta()
    #     .reproject(crs=landsat_img.select(['B2']).projection().crs(), scale=landsat_cs)\
    #     .reduceResolution(reducer=ee.Reducer.mean(), maxPixels=65535) \
    #     .reproject(crs=output_crs, crsTransform=output_geo) \
    #     .updateMask(1)

asset_id = '{coll}_{cs}m/{image}'.format(
    coll='projects/disalexi/ta/landsat', image=export_id, cs=export_cs)
task_id = 'disalexi_tair_landsat_{image}_{cs}m'.format(image=export_id, cs=export_cs)

ta_landsat_img = ta_landsat_func(export_img)\
    .setMulti({
        'DATE_INGESTED': datetime.datetime.now().strftime('%Y-%m-%d'),
        'DISALEXI_VERSION': openet.disalexi.__version__,
        'DATE': ee.Date(export_img.get('system:time_start')).format('YYYY-mm-dd'),
    })
task = ee.batch.Export.image.toAsset(
    image=ee.Image(ta_landsat_img).toFloat(),
    description=task_id,
    assetId=asset_id,
    crs=landsat_crs,
    scale=30
    # crsTransform='[' + ','.join(list(map(str, export_geo))) + ']',
    # dimensions='{0}x{1}'.format(*export_shape),
)
# task.start()
# time.sleep(1)
# print(task.status())

### Thumbnails

In [10]:
# # thumbnail_crs = 'EPSG:4326'
# # thumbnail_cs = 0.005
# thumbnail_crs = 'EPSG:32610'
# thumbnail_cs = 120
# thumbnail_xy = output_geom.bounds(1, thumbnail_crs).coordinates().getInfo()[0]
# thumbnail_xmin = int(min(x for x, y in thumbnail_xy) / thumbnail_cs) * thumbnail_cs
# thumbnail_ymin = int(min(y for x, y in thumbnail_xy) / thumbnail_cs) * thumbnail_cs
# thumbnail_xmax = int(max(x for x, y in thumbnail_xy) / thumbnail_cs) * thumbnail_cs + thumbnail_cs
# thumbnail_ymax = int(max(y for x, y in thumbnail_xy) / thumbnail_cs) * thumbnail_cs + thumbnail_cs
# thumbnail_geo = [thumbnail_cs, 0.0, thumbnail_xmin, 0.0, thumbnail_cs, thumbnail_ymax]
# thumnbail_region = [[], [], [], []]
# thumbnail_shape_str = '{0}x{1}'.format(
#     int(abs(thumbnail_xmax - thumbnail_xmin) / thumbnail_cs),
#     int(abs(thumbnail_ymax - thumbnail_ymin) / thumbnail_cs))
# print(thumbnail_crs)
# print(thumbnail_geo)
# print(thumbnail_shape_str)

In [11]:
# landsat_url = landsat_img.select(['B4', 'B3', 'B2'])\
#     .reproject(crs=thumbnail_crs, crsTransform=thumbnail_geo)\
#     .getThumbURL({'region': output_region, 'min': 0, 'max': 0.30})
# # print(landsat_url)
# Image(url=landsat_url)

NameError: name 'thumbnail_crs' is not defined

In [ ]:
# landsat_url = landsat_coll.filterDate('2015-07-11', '2015-07-12')\
#     .median().select(['B4', 'B3', 'B2'])\
#     .reproject(crs=thumbnail_crs, crsTransform=thumbnail_geo)\
#     .getThumbURL({'region': output_region, 'min': 0, 'max': 0.30})
# # print(landsat_url)
# Image(url=landsat_url)

In [ ]:
# landsat_url = landsat_coll.filterDate('2015-07-11', '2015-07-12')\
#       .mean().select(['B4', 'B3', 'B2'])\
#       .reproject(crs=output_crs, crsTransform=output_geo)\
#       .getThumbURL({'region': output_region, 'min': 0, 'max': 0.30})
#     # print(landsat_url)
# Image(url=landsat_url)

### Ta Image

In [ ]:
# ta_landsat_url = ta_landsat_func(landsat_img)\
#     .reproject(crs=thumbnail_crs, crsTransform=thumbnail_geo)\
#     .getThumbURL({
#         'region': output_region, 'min': 273, 'max': 325, 
#         'palette': ','.join(['FF0000', 'FFFF00', '00FFFF', '0000FF'])})
# print(ta_landsat_url)
# Image(url=ta_landsat_url)

In [ ]:
# ta_coarse_url = ta_coarse_func(landsat_img)\
#     .reproject(crs=thumbnail_crs, crsTransform=thumbnail_geo)\
#     .getThumbURL({'region': output_region, 'min': 273, 'max': 325, 
#                   'palette': ','.join(['FF0000', 'FFFF00', '00FFFF', '0000FF'])})
# print(ta_coarse_url)
# Image(url=ta_coarse_url)